In [24]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import re
import torch
from time import perf_counter
import tabulate
import pandas as pd

In [25]:
select = 'ohsumed'
dtype = 'train'
with open('data/ind.' + select + '.' + dtype + '.x', 'rb') as f:
    data = pkl.load(f)

In [44]:
len(data)

3022

In [43]:
data

[4650,
 5437,
 6229,
 4322,
 6512,
 4428,
 7012,
 5810,
 4850,
 5649,
 6573,
 6656,
 7213,
 6183,
 7308,
 5498,
 5983,
 6564,
 7269,
 4450,
 5978,
 6911,
 5338,
 5417,
 5151,
 4978,
 7036,
 4296,
 4762,
 6219,
 5770,
 6284,
 5859,
 6475,
 4237,
 4257,
 5714,
 5675,
 4571,
 4629,
 6667,
 4186,
 5850,
 5628,
 7130,
 4529,
 6337,
 5769,
 7380,
 4144,
 5110,
 4222,
 4153,
 4982,
 6117,
 4748,
 4800,
 6574,
 6084,
 4252,
 5692,
 5933,
 7121,
 4496,
 6831,
 5438,
 6270,
 4686,
 5900,
 6280,
 5910,
 4779,
 4773,
 6057,
 7117,
 4958,
 6644,
 6228,
 4489,
 6023,
 4634,
 4159,
 7175,
 6120,
 6759,
 4498,
 6262,
 7278,
 6458,
 6301,
 4373,
 7137,
 5706,
 7151,
 6214,
 6089,
 4631,
 7084,
 5966,
 5345,
 4062,
 6764,
 4830,
 5618,
 7138,
 4151,
 4217,
 5389,
 5845,
 6304,
 4976,
 6124,
 4969,
 4577,
 5277,
 5883,
 7329,
 5179,
 4713,
 5827,
 5542,
 6440,
 6276,
 4684,
 4796,
 5040,
 5605,
 6268,
 5171,
 5750,
 6568,
 6311,
 6104,
 5453,
 5182,
 5300,
 4556,
 7054,
 7108,
 7388,
 6998,
 7279,
 6523,

In [36]:
lda_feat = pd.read_csv('../../../tp_data_all/' + select + '_topic_modelled_final_data.csv')

In [48]:
len(list(lda_feat.iloc[0][2:]))

23

In [49]:
lda_feat

,content,labels,0,1,2,3,4,5,6,7,...,13,14,15,16,17,18,19,20,21,22
0,treating diabetes elderly special consideratio...,C19,0.004419,0.003920,0.008245,0.002483,0.006160,0.005703,0.428260,0.004156,...,0.010160,0.003886,0.049664,0.141783,0.004551,0.005032,0.143174,0.081538,0.001569,0.004546
1,antidiabetic agents although either insulin or...,C19,0.000907,0.026014,0.001695,0.032059,0.001263,0.006688,0.395556,0.014000,...,0.002236,0.013952,0.067291,0.074077,0.000934,0.005405,0.058926,0.194099,0.000322,0.000937
2,effects repetitive administration corticotropi...,C19,0.001040,0.000923,0.001941,0.005578,0.006458,0.001343,0.113022,0.011028,...,0.002469,0.005918,0.021734,0.118329,0.001071,0.001184,0.047297,0.599261,0.000369,0.001074
3,minimal model analysis intravenous glucose tol...,C19,0.000949,0.000842,0.058855,0.000533,0.001321,0.001224,0.109774,0.000893,...,0.002276,0.000857,0.332322,0.015852,0.016880,0.001080,0.041491,0.138854,0.000337,0.000982
4,advanced protein glycosylation induces transen...,C19,0.001296,0.001074,0.002287,0.000680,0.019270,0.001562,0.322887,0.150163,...,0.078854,0.001064,0.037028,0.028367,0.062915,0.013085,0.099208,0.112311,0.000430,0.015808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,use prophylactic furazolidone control nosocomi...,C01,0.002496,0.062636,0.004816,0.001403,0.003476,0.003222,0.232764,0.002348,...,0.030783,0.002195,0.003888,0.165100,0.002571,0.002843,0.070024,0.119990,0.000886,0.002573
3353,randomized prospective field trial conjugate v...,C01,0.001630,0.009318,0.010929,0.000916,0.002269,0.002103,0.211572,0.001533,...,0.037319,0.001433,0.010419,0.115610,0.017728,0.033572,0.213863,0.147955,0.000579,0.001678
3354,tampon absorbency composition oxygen content r...,C01,0.003095,0.002745,0.005774,0.001739,0.004309,0.003994,0.375778,0.002910,...,0.007351,0.002721,0.004819,0.029486,0.003187,0.003524,0.303220,0.156896,0.001099,0.003184
3355,amoxycillin plus probenecid versus doxycycline...,C01,0.004152,0.003683,0.027845,0.002333,0.005782,0.005358,0.344011,0.003905,...,0.009799,0.003651,0.006466,0.173009,0.004276,0.044921,0.030165,0.146933,0.001474,0.004276
